<a href="https://colab.research.google.com/github/ShovalBenjer/Football-Stats-Analysis-Web-Scraping-KPI-Extraction-and-Machine-Learning/blob/main/Football_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install opencv-python opencv-python-headless

In [3]:
import cv2
import numpy as np
import os

# Paths
video_path = r"C:\Users\shova\Downloads\משחק האירופאיות _ המשחק המלא_ מכבי חיפה - הפועל ב_ש 0-3.mp4"
yolo_weights = r"path_to_yolov3.weights"
yolo_config = r"path_to_yolov3.cfg"
coco_names = r"path_to_coco.names"

# Load YOLO
net = cv2.dnn.readNet(yolo_weights, yolo_config)
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

# Load COCO class names
with open(coco_names, 'r') as f:
    classes = [line.strip() for line in f.readlines()]

# Function to check aspect ratio
def is_standard_aspect_ratio(frame, expected_ratio=16/9, tolerance=0.1):
    height, width, _ = frame.shape
    aspect_ratio = width / height
    return abs(aspect_ratio - expected_ratio) < tolerance

# Function to detect objects
def detect_objects(frame):
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    detections = net.forward(output_layers)

    objects_detected = []
    height, width, _ = frame.shape

    for detection in detections:
        for object_detected in detection:
            scores = object_detected[5:]
            class_id = int(np.argmax(scores))
            confidence = scores[class_id]
            if confidence > 0.5:  # Confidence threshold
                center_x = int(object_detected[0] * width)
                center_y = int(object_detected[1] * height)
                w = int(object_detected[2] * width)
                h = int(object_detected[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                objects_detected.append({
                    "class_id": class_id,
                    "label": classes[class_id],
                    "confidence": float(confidence),
                    "box": [x, y, w, h]
                })

                # Draw bounding box (optional)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.putText(frame, f"{classes[class_id]}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame, objects_detected

# Video processing and filtering pipeline
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    saved_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        if not is_standard_aspect_ratio(frame):
            continue

        processed_frame, detections = detect_objects(frame)

        # Filter based on detections
        if len(detections) > 0:
            # Save or further process the frame
            saved_frames += 1
            cv2.imshow("Frame", processed_frame)
            key = cv2.waitKey(1)
            if key == ord('q'):  # Press 'q' to quit early
                break

        if frame_count > 100:  # Process only first 100 frames for PoC
            break

    cap.release()
    cv2.destroyAllWindows()
    print(f"Processed {frame_count} frames, saved {saved_frames} relevant frames.")

# Run the PoC
process_video(video_path)


error: OpenCV(4.10.0) /io/opencv/modules/dnn/src/darknet/darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: path_to_yolov3.cfg in function 'readNetFromDarknet'
